<a href="https://colab.research.google.com/github/corentinBlanchard/chatbot-IA/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
downloaded = drive.CreateFile({'id':'1asz0LQpmjqAfx7_rc6WxkOeYOS848F2S'}) # replace the id with id of file you want to access
dopwnloaded = drive.
downloaded.GetContentFile('ia.yml') 

In [19]:
import yaml
file = open('ia.yml')
docs = yaml.safe_load(file)
print(docs['conversations'][0])

['What is AI?', 'Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.']
